In [2]:
%matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import matplotlib 
from PIL import Image
import PIL
import math
from random import random
from random import seed
from random import randint
import time
from numba import jit
import skimage

Using matplotlib backend: Qt5Agg


In [91]:
@jit(nopython = True)
    
def Laplacian(A,deltax,deltat):

    h = len(A)
    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    P = np.zeros((h,h))
    result = np.zeros((h,h))
    
    for i in range(0, h):
        for j in range(0,h):
            P[i,j] = (-A[(i+2)%h,j] + 16*A[(i+1)%h,j] + 16*A[(i-1)%h,j] -A[(i-2)%h,j] \
                 -A[i,(j+2)%h] + 16*A[i,(j+1)%h]  + 16*A[i,(j-1)%h] -A[i,(j-2)%h] - 60*A[i,j])/(12*pow(deltax,2))

  
    return P 

def damping(t,K_damp):
    Damping = np.zeros(shape=int(t/deltat))
    Damping[0:200] = 1*K_damp
    return Damping

def div_potential_1(A1,A2,omega,eta):
    lam = 2*pow(math.pi,2)/pow(omega,2)
    pot = lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2))
    return pot

def div_potential_2(A1,A2,omega,eta):
    lam = 2*pow(math.pi,2)/pow(omega,2)
    pot = lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2))
    return pot
    
def function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping):
    F = np.subtract(Laplacian(A1,deltax,deltat),div_potential_1(A1,A2,omega,eta)) - Damping[int(index)]*vel1
    return F

def function_2(A1,A2,vel2,omega,eta,deltax,deltat,index,Damping):
    F = np.subtract(Laplacian(A2,deltax,deltat),div_potential_2(A1,A2,omega,eta)) - Damping[int(index)]*vel2
    return F

def update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,index,Damping):
    F1 = function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping)
    F2 = function_2(A1,A2,vel2,omega,eta,deltax,deltat,index,Damping)
    
    B1 = A1
    
    A1 = A1 + deltat*(vel1 + 0.5*deltat*function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping)) 
    A2 = A2 + deltat*(vel2 + 0.5*deltat*function_2(B1,A2,vel2,omega,eta,deltax,deltat,index,Damping)) 
    
    vel1 = vel1 + deltat/2*(F1 + function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping)) 
    vel2 = vel2 + deltat/2*(F2 + function_2(A1,A2,vel2,omega,eta,deltax,deltat,index,Damping))
    return A1,A2,vel1,vel2
    
#change the values accordlingly    
def wolff(N,t,A1,A2,vel,omega,eta,deltax,deltat,K_damp):

    Damping = damping(t,K_damp)

    vel1 = vel
    vel2 = vel
    
    fig, ax = plt.subplots(1, 2,figsize=(10,10))

    #ax[0].axis('off')
    #ax[1].axis('off')
    ax[0].get_xaxis().set_visible(False)
    ax[0].get_yaxis().set_visible(False)
    ax[1].get_xaxis().set_visible(False)
    ax[1].get_yaxis().set_visible(False)
    
    plt.ion()
    plt.show()
    cbax = fig.add_axes([0.04,0.1,0.03,0.8])
    cbay = fig.add_axes([0.93,0.1,0.03,0.8])

    
    for i in np.arange(int(t/deltat)):
        

        
        if i == 0:
            first = ax[0].imshow(np.sqrt(pow(A1,2)+pow(A2,2)), vmin= 0, vmax=1.5*round(eta),cmap = 'Greys', interpolation = 'none')
            fig.colorbar(first,orientation = 'vertical',cax = cbax)
            second = ax[1].imshow(np.arctan2(A2,A1), vmin= -1*math.pi, vmax=math.pi,cmap = 'twilight', interpolation = 'none')
            fig.colorbar(second,orientation = 'vertical',cax = cbay)
            
        A1,A2,vel1,vel2 = update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,i,Damping)
        #print([A1[0,0],A2[0,0]])
        
        

        
        if i in range(50) or i % 100 == 0 and i != 0:
            ax[0].imshow(np.sqrt(pow(A1,2)+pow(A2,2)), vmin= 0, vmax=1.5*round(eta),cmap = 'Greys',interpolation = 'none')
  
            ax[1].imshow(np.arctan2(A2,A1), vmin= -1*math.pi, vmax=math.pi,cmap = 'twilight', interpolation = 'none')

            plt.pause(1)
            plt.suptitle('time= '+str(round((i*deltat),1))+'')

    return A1,A2


In [92]:
K_damp = 1
n = 5
deltax = 1
deltat = 0.1
eta = 1
N = pow(2,n)
t = 0.5*N*deltax/deltat # light crossing time
#t10 = t/10 # 1/10 of the ligth crossing time
vel = np.zeros((N,N))
mu, sigma = 0, 0.1 # mean and standard deviation
omega = 5

A1 = np.random.normal(mu, sigma, size=(N, N))
A2 = np.random.normal(mu,sigma,size=(N,N))
#A = np.random.randn(N, 2*N).view(np.complex128)   

A1,A2 = wolff(N,t,A1,A2,vel,omega,eta,deltax,deltat,K_damp)

In [5]:
Laplacian(np.zeros((10,10)),1,0.1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])